In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns 
import time
from datetime import date

# ROUND 2

In [ ]:
# Instructions

# Show the dataframe shape.
# Standardize header names.
# Which columns are numerical?
# Which columns are categorical?
# Check and deal with NaN values.
# Datetime format - Extract the months from the dataset and store in a separate column. 
# Then filter the data to show only the information for the first quarter , ie. January, February and March. 
# Hint: If data from March does not exist, consider only January and February.

In [ ]:
data = pd.read_csv('marketing_customer_analysis.csv')

data = data.drop(['Unnamed: 0'],axis=1)

data # This also shows the shape (10910 x 25)

In [ ]:
print(data.columns) # Look for standardized headers. The dataset headers seem okay already? Perhaps lowercase?

cols = []
for i in range(len(data.columns)):
    cols.append(data.columns[i].lower().replace(' ','_')) # I've added the snake, using .replace('', '_')
data.columns = cols

data # To confirm the for loop worked on the headers

In [ ]:
print(data.dtypes)

# Categorical: All columns with 'object' type

# Numerical: All columns with 'float64' and 'int64' types

# Here we can also pull out numerical and categorical columns and filter / print them 

# for example: isNumeric = is_numeric(df)

In [ ]:
sum(data.duplicated()) # check for duplicates

data.drop_duplicates(inplace=True) # remove duplicates

sum(data.duplicated()) # confirm that duplicates have been removed

In [ ]:
# Looking at data for how to address null values

data.isna().sum()

In [ ]:
round(data.isna().sum()/len(data),4)*100  # shows the percentage of null values in a column
nulls_df = pd.DataFrame(round(data.isna().sum()/len(data),4)*100)
nulls_df
nulls_df = nulls_df.reset_index()
nulls_df
nulls_df.columns = ['header_name', 'percent_nulls']
nulls_df

# Based on the outputs of null_df, I would prefer to remove the 'vehicle class' column altogether and drop rows with >0 nulls 
# (It's a large dataset so should not introduce any bias)

In [ ]:
data['state'].value_counts() # Judging by this quick look at the data, probably best to just remove rows with NaN than to replace with data (especially since it is categorical)

In [ ]:
data_clean = data = data.drop('vehicle_type', axis=1) # First remove this column so the next command does not drop about 5,000 rows from the data set

data_clean = data.dropna(axis=0) # This is the simplest and most effective to remove all rows with NaN 

data_clean.isna().sum() # Confirm NaNs are removed from the dataset

In [ ]:
data_clean.reset_index(drop=True) # Make sure the new 'clean' dataset looks okay before proceeding

In [ ]:
data_clean['effective_to_date'] = pd.to_datetime(data_clean['effective_to_date'], errors='coerce')

data_clean['month'] = pd.DatetimeIndex(data_clean['effective_to_date']).month

data_clean

#data_clean.sort_values('effective to date') # Based on the df, it already only includes data from January and February??

In [ ]:
# extract months from dataset and create new column

data_clean_Q1 = data_clean[(data_clean['month']<4)]

data_clean_Q1 # Confirming that only Q1 data is in the new dataset

In [ ]:
# BONUS - Function for all cleaning / pre-processing

...


# ROUND 3

In [ ]:
# Instructions 

# EDA (Exploratory Data Analysis) - Complete the following tasks to explore the data:
# Show DataFrame info.
# Describe DataFrame.
# Show a plot of the total number of responses.
# Show a plot of the response rate by the sales channel.
# Show a plot of the response rate by the total claim amount.
# Show a plot of the response rate by income.

In [ ]:
pd.set_option('display.max_columns', None)
data_clean.info()

In [ ]:
data_clean.describe().T 

In [ ]:
response_count = data_clean['response'].value_counts()

response_count.plot(kind='bar')

In [ ]:
# First option for sales channel

# data_clean.groupby('sales channel')['response_count'].mean().plot.bar()

# plt.show()

# Other option for sales channel... sns.countplot()

In [ ]:
data_clean['response_binary'] = data_clean['response'].map({'Yes': 1, 'No': 0})

data_clean.reset_index(drop=True) 

data_clean.head(10) # Test to make sure the map function passed the dictionary to the new list response_binary

In [ ]:
sales_channel_response_rate = data_clean.groupby('response_binary')['sales_channel'].value_counts(normalize=True).unstack('sales channel')

sales_channel_response_rate

sales_channel_response_rate.plot(kind='bar')


In [ ]:
sns.barplot(x='response_binary', y='total_claim_amount', data=data_clean)
plt.show() 

In [ ]:
sns.barplot(x='response_binary', y='income', data=data_clean)
plt.show() 

# ROUND 4

In [76]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
data_clean.info()

In [114]:
data_num = data_clean.select_dtypes(include= np.number)

data_num = data_num.drop(['response_binary'], axis=1)

data_cat = data_clean.select_dtypes(include= object)

In [ ]:
sns.displot(data_num['income']) # Example of one column distribution plot...
plt.show()

In [ ]:
# Plotting all of the columns in data_num all at once

for i, column in enumerate(data_num.columns, 1):
    sns.displot(data_num[column])

# Not all of the variable plots are normally distributed. 
# For instance, month is bimodal (by design)
# months_since_policy_inception may be normally distributed.

In [ ]:
# Now using Matplotlib - not a best practice to pass same number of bins for all variables 
# But this is the simplest way I could find to do this

data_num.hist(bins=25, figsize=(30, 25))

In [ ]:
# Checking for multicollinearity 

correlations_matrix = data_num.corr()
sns.set(rc = {'figure.figsize':(10,5)})
sns.heatmap(correlations_matrix, annot=True)
plt.show()

In [ ]:
# Check why total claim amount and income have a moderate negative correlation 
# (just out of curiosity).

sns.scatterplot(x='total_claim_amount', y='income', data=data_num)
plt.show()

In [ ]:
# Should we be removing the outliers at this stage?

# ROUND 5

In [124]:
from sklearn import linear_model
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import MinMaxScaler 

In [125]:
y = data_clean['total_claim_amount']
X = data_num.drop(['total_claim_amount'], axis=1)
# X_num = X.select_dtypes(include = np.number). Saving these for later...
# X_cat = X.select_dtypes(include = object)

In [ ]:
lm = linear_model.LinearRegression()

lm.fit(X,y)
print(lm.score(X,y))
y_pred = lm.predict(X)
print(mean_squared_error(y_pred,y))

In [ ]:
print(lm.coef_) 

# Checking coefficients. Need to know p values for each to test significance...

In [ ]:
transformer = MinMaxScaler().fit(X)
x_normalized = transformer.transform(X)
print(x_normalized.shape)
x_normalized

# Use MinMaxScaler to normalize the data

In [130]:
data_num_normalized = pd.DataFrame(x_normalized, columns=X.columns) # pass this into a df so we can 

In [ ]:
data_num_normalized.describe().T

In [ ]:
sns.regplot(x=X['income'], y=y)

# Just visualising one of the relationships to see what it looks like.